# Preprocesamiento

In [ ]:
import os
import numpy as np
import pandas as pd
import re
from typing import List

In [ ]:
os.chdir("..")
os.getcwd()

'c:\\Users\\fscasso\\OneDrive - S.A. La Nacion\\Documentos\\repos\\itba_mlops'

In [75]:
RAW_FD = "raw"
PREPROC_FD = "data"

In [ ]:
peliculas = pd.read_csv(DATA_URL + "peliculas_0.csv")
personas = pd.read_csv(DATA_URL + "personas_0.csv")
scores = pd.read_csv(DATA_URL + "scores_0.csv")
trabajadores = pd.read_csv(DATA_URL + "trabajadores_0.csv")
usuarios = pd.read_csv(DATA_URL + "usuarios_0.csv")

In [ ]:
peliculas.to_csv(os.path.join(RAW_FD, "peliculas_0.csv"), index=False)
personas.to_csv(os.path.join(RAW_FD, "personas_0.csv"), index=False)
scores.to_csv(os.path.join(RAW_FD, "scores_0.csv"), index=False)
trabajadores.to_csv(os.path.join(RAW_FD, "trabajadores_0.csv"), index=False)
usuarios.to_csv(os.path.join(RAW_FD, "usuarios_0.csv"), index=False)
del peliculas, personas, scores, trabajadores, usuarios

## Películas

In [23]:
peliculas = pd.read_csv(os.path.join(RAW_FD, "peliculas_0.csv"))
peliculas.head()

,index,id,Name,Release Date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,4,Get Shorty (1995),1995-01-01,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,5,Copycat (1995),1995-01-01,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [24]:
# Chequeos
assert (peliculas.index == peliculas["index"]).all()
assert (peliculas.index + 1 == peliculas.id).all()

aux = peliculas.copy()
aux = aux.drop(["index", "id", "Name", "Release Date", "IMDB URL"], axis=1)
for c in aux.columns:
    assert set(aux[c].unique()) == {0, 1}  # para bools

In [25]:
# Cambios al df
peliculas = peliculas.drop("index", axis=1)
peliculas = peliculas.set_index("id", drop=True, verify_integrity=True)

In [26]:
def order_cols(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    return df[cols + [c for c in df.columns if c not in cols]]

In [27]:
peliculas.head()

,Name,Release Date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),1995-01-01,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),1995-01-01,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [29]:
peliculas["release_date"] = pd.to_datetime(peliculas["Release Date"])

peliculas["Name"] = peliculas.Name.str.strip()

YEAR_PATT = re.compile(r"[1-2]\d{3}$")

peliculas["year"] = peliculas.Name.str[-5:-1]
peliculas["year_ok"] = peliculas.year.map(lambda v: YEAR_PATT.match(v))
peliculas["year"] = peliculas.apply(lambda row: float(row.year) if row.year_ok else np.nan, axis=1)

peliculas["name"] = peliculas.apply(lambda row: row.Name[:-7] if row.year_ok else row.Name, axis=1)
peliculas = peliculas.drop("year_ok", axis=1)

peliculas["imdb_url"] = peliculas["IMDB URL"].str[19:]

peliculas = peliculas.drop(["Release Date", "Name", "IMDB URL"], axis=1)

META_COLS = ["name", "year", "release_date", "imdb_url"]
GENRE_COLS = [c for c in peliculas.columns if c not in META_COLS]
peliculas = order_cols(peliculas, META_COLS)

peliculas = peliculas.rename({c: c.lower() for c in GENRE_COLS}, axis=1)
peliculas = peliculas.rename({"children's": "childrens", "film-noir": "film_noir", "sci-fi": "sci_fi"}, axis=1)
GENRE_COLS = [c for c in peliculas.columns if c not in META_COLS]

peliculas = peliculas.astype({c: bool for c in GENRE_COLS})

peliculas.head()

,name,year,release_date,imdb_url,unknown,action,adventure,animation,childrens,comedy,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story,1995.0,1995-01-01,M/title-exact?Toy%20Story%20(1995),False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,GoldenEye,1995.0,1995-01-01,M/title-exact?GoldenEye%20(1995),False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,Four Rooms,1995.0,1995-01-01,M/title-exact?Four%20Rooms%20(1995),False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,Get Shorty,1995.0,1995-01-01,M/title-exact?Get%20Shorty%20(1995),False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
5,Copycat,1995.0,1995-01-01,M/title-exact?Copycat%20(1995),False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [30]:
peliculas.columns

Index(['name', 'year', 'release_date', 'imdb_url', 'unknown', 'action',
       'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary',
       'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery',
       'romance', 'sci_fi', 'thriller', 'war', 'western'],
      dtype='object')

In [20]:
# peliculas[~(peliculas.year == peliculas.release_date.dt.year)]  # 110 casos en que el anio no es el mismo

In [31]:
peliculas[peliculas.name == "unknown"]

,name,year,release_date,imdb_url,unknown,action,adventure,animation,childrens,comedy,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
id,,,,,,,,,,,,,,,,,,,,,
267,unknown,NaN,NaT,NaN,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Personas

In [51]:
personas = pd.read_csv(os.path.join(RAW_FD, "personas_0.csv"))
personas

,index,id,Full Name,year of birth,Gender,Zip Code
0,0,1,Robert Stanley,1974,M,85711
1,1,2,Heather Bossey,1945,F,94043
2,2,3,Don Davis,1975,M,32067
3,3,4,Daniel Mccauley,1974,M,43537
4,4,5,Janice Mccullough,1965,F,15213
...,...,...,...,...,...,...
748,748,749,Justin Petersen,1965,M,80919
749,749,750,Brock Vasquez,1970,M,32303
750,750,751,Norma Penix,1974,F,90034
751,751,752,Ralph Maugeri,1938,M,21201


In [52]:
assert (personas.index == personas["index"]).all()
assert (personas.index + 1 == personas.id).all()

personas = personas.drop("index", axis=1)
personas = personas.set_index("id", drop=True, verify_integrity=True)

personas = personas.rename(
    {"year of birth": "year_birth", "Gender": "gender", "Zip Code": "zip_code"},
    axis=1
)
personas = personas.astype({"year_birth": int})

assert personas.gender.nunique() == 2
personas["is_female"] = personas.gender == "F"
personas = personas.drop("gender", axis=1)

assert (personas.zip_code.str.len() == 5).all()  # aunque usan letras y numeros a piacere

personas.head()

,Full Name,year_birth,zip_code,is_female
id,,,,
1,Robert Stanley,1974,85711,False
2,Heather Bossey,1945,94043,True
3,Don Davis,1975,32067,False
4,Daniel Mccauley,1974,43537,False
5,Janice Mccullough,1965,15213,True


In [54]:
# personas.gender.value_counts()
# personas.zip_code.value_counts()
personas.zip_code.sort_values()
# personas.year_birth.sort_values()

id
186    00000
9      01002
39     01040
231    01080
55     01331
       ...  
719    V0R2H
333    V0R2M
145    V3N4P
490    V5A2B
458    Y1A6B
Name: zip_code, Length: 753, dtype: object

## Scores

In [60]:
scores = pd.read_csv(os.path.join(RAW_FD, "scores_0.csv"))
scores

,index,Unnamed: 0,user_id,movie_id,rating,Date
0,16668,16668,93,235,4,1998-02-28 22:45:39
1,2626,2626,210,58,4,1998-02-17 15:42:57
2,24623,24623,85,28,4,1997-11-18 05:01:41
3,11406,11406,211,357,2,1997-11-13 22:29:32
4,18828,18828,271,272,3,1998-01-26 19:56:23
...,...,...,...,...,...,...
24995,10734,10734,64,161,3,1998-03-12 21:56:19
24996,44280,44280,40,321,4,1998-03-04 19:58:43
24997,79240,79240,108,1,4,1997-11-18 19:02:00
24998,31783,31783,311,639,4,1998-01-09 17:08:06


In [61]:
assert (scores["index"] == scores["Unnamed: 0"]).all()

In [62]:
scores = scores.drop("Unnamed: 0", axis=1)
scores = scores.set_index("index", drop=True, verify_integrity=True)
scores = scores.rename({"Date": "fecha_hora"}, axis=1)
scores

,user_id,movie_id,rating,fecha_hora
index,,,,
16668,93,235,4,1998-02-28 22:45:39
2626,210,58,4,1998-02-17 15:42:57
24623,85,28,4,1997-11-18 05:01:41
11406,211,357,2,1997-11-13 22:29:32
18828,271,272,3,1998-01-26 19:56:23
...,...,...,...,...
10734,64,161,3,1998-03-12 21:56:19
44280,40,321,4,1998-03-04 19:58:43
79240,108,1,4,1997-11-18 19:02:00


In [66]:
scores.describe()

,user_id,movie_id,rating
count,25000.000000,25000.000000,25000.000000
mean,194.317760,277.999880,3.673600
std,112.534946,179.251157,1.104605
min,1.000000,1.000000,1.000000
25%,92.000000,134.000000,3.000000
50%,206.000000,248.000000,4.000000
75%,296.000000,419.000000,5.000000
max,377.000000,672.000000,5.000000


In [65]:
scores.rating.value_counts().sort_index()

rating
1    1243
2    2333
3    6206
4    8777
5    6441
Name: count, dtype: int64

## Join

In [70]:
scores.head()
# peliculas.head()
# personas.head()

,user_id,movie_id,rating,fecha_hora
index,,,,
16668,93,235,4,1998-02-28 22:45:39
2626,210,58,4,1998-02-17 15:42:57
24623,85,28,4,1997-11-18 05:01:41
11406,211,357,2,1997-11-13 22:29:32
18828,271,272,3,1998-01-26 19:56:23


In [72]:
df = scores.merge(peliculas.reset_index(), how="inner", left_on="movie_id", right_on="id")
df = df.merge(personas.reset_index(), how="inner", left_on="user_id", right_on="id")
df = df.drop(["id_x", "id_y"], axis=1)
df

,user_id,movie_id,rating,fecha_hora,name,year,release_date,imdb_url,unknown,action,...,mystery,romance,sci_fi,thriller,war,western,Full Name,year_birth,zip_code,is_female
0,93,235,4,1998-02-28 22:45:39,Mars Attacks!,1996.0,1996-12-13,M/title-exact?Mars%20Attacks!%20(1996),False,True,...,False,False,True,False,True,False,Jeffrey Sheets,1950,23112,False
1,93,121,3,1998-02-28 22:30:53,Independence Day (ID4),1996.0,1996-07-03,M/title-exact?Independence%20Day%20(1996),False,True,...,False,False,True,False,True,False,Jeffrey Sheets,1950,23112,False
2,93,222,4,1998-02-28 22:34:55,Star Trek: First Contact,1996.0,1996-11-22,M/title-exact?Star%20Trek:%20First%20Contact%2...,False,True,...,False,False,True,False,False,False,Jeffrey Sheets,1950,23112,False
3,93,118,3,1998-02-28 22:36:56,Twister,1996.0,1996-05-10,M/title-exact?Twister%20(1996),False,True,...,False,False,False,True,False,False,Jeffrey Sheets,1950,23112,False
4,93,1,5,1998-02-28 22:35:21,Toy Story,1995.0,1995-01-01,M/title-exact?Toy%20Story%20(1995),False,False,...,False,False,False,False,False,False,Jeffrey Sheets,1950,23112,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,36,310,4,1997-12-15 03:42:07,"Rainmaker, The",1997.0,1997-01-01,"M/title-exact?Rainmaker,+The+(1997)",False,False,...,False,False,False,False,False,False,Maria Kaplan,1979,93117,True
24996,36,261,5,1997-12-15 03:46:21,Air Bud,1997.0,1997-08-01,M/title-exact?Air+Bud+(1997),False,False,...,False,False,False,False,False,False,Maria Kaplan,1979,93117,True
24997,36,319,2,1997-12-15 03:42:36,Everyone Says I Love You,1996.0,1996-12-06,M/title-exact?Everyone%20Says%20I%20Love%20You...,False,False,...,False,True,False,False,False,False,Maria Kaplan,1979,93117,True
24998,36,307,4,1997-12-15 03:40:27,"Devil's Advocate, The",1997.0,1997-01-01,"M/title-exact?Devil's+Advocate,+The+(1997)",False,False,...,True,False,False,True,False,False,Maria Kaplan,1979,93117,True


In [76]:
peliculas.to_csv(os.path.join(PREPROC_FD, "movies.csv"))
personas.to_csv(os.path.join(PREPROC_FD, "users.csv"))
scores.to_csv(os.path.join(PREPROC_FD, "scores.csv"))